# Transaction Sequencing & Batch creation

### Import statements and data loading

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np  
import tensorflow as tf
from typing import Optional

df = pd.read_csv('../../datasets/modified/modified_sparkov_full.csv', index_col='Unnamed: 0')
df.head()

2023-03-15 16:51:04.992213: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 16:51:05.410048: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-15 16:51:05.410090: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-15 16:51:07.358234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

,cc_num,merchant,category,amt,gender,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,age,trans_timedelta,trans_month_sin,trans_month_cos,trans_week_sin,trans_week_cos,trans_hour_sin,trans_hour_cos
0,146,585,12,-0.408741,1,29.0,93.0,-0.282429,1325376018,49.0,25.0,0,30,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
1,51,105,2,0.233378,1,62.0,50.0,-0.293527,1325376044,57.0,79.0,0,40,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
2,860,366,7,0.942184,-1,69.0,95.0,-0.280243,1325376051,63.0,73.0,0,56,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
3,696,39,1,-0.157381,-1,37.0,66.0,-0.287590,1325376076,7.0,72.0,0,51,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
4,195,521,11,-0.176470,-1,62.0,79.0,-0.293693,1325376186,58.0,93.0,0,32,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818


It is mportant things to note that the dataset is **sorted by credit card number and then by chronological order**

---

### Creating intervaled sequences of transactions

To create sequences, we need:
1. Create a new Dataframe to host the new dataset format
2. Iterate through the original dataset
3. With a cutoff length of **100**, each row of the new dataset will contain 100 transaction, in chronological order, of a given card

### Creating sequences

##### Dividing into train, validation and test datasets
training, validation and test datasets are splt based on the time period they are placed in, to simulate real life

In [2]:
# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size = 0.8
valid_size=0.1
len_cc = 1000
cutoff_length = 6

train_index = int(len(df)*train_size)

df_train = df[0:train_index]
df_rem = df[train_index:]

valid_index = int(len(df)*valid_size)

df_valid = df[train_index:train_index+valid_index]
df_test = df[train_index+valid_index:]

In [ ]:
from itertools import dropwhile

df_list = [df_train, df_valid, df_test]
fraud_seq_list = [{}, {}, {}]
non_fraud_seq_list = [{}, {}, {}]

for set_id in range(len(df_list)):
    set = df_list[set_id].groupby(['cc_num'])
    for name, group in set:
        fraud_seq_list[set_id][name] = [] # to initialize empty list of sequences
        non_fraud_seq_list[set_id][name] = [] 
        
        sequence_list = group.rolling(window=cutoff_length)#
        for sequence in dropwhile(lambda w: len(w) < cutoff_length, group.rolling(window=cutoff_length)):
            sequence = sequence.reset_index(drop=True)
            if (sequence.tail(1)["is_fraud"] == 1).bool():
                fraud_seq_list[set_id][name] += [sequence.to_numpy()]
            else:
                non_fraud_seq_list[set_id][name] += [sequence.to_numpy()]


fraud_seq_list

### Creating Batches

The dataset needs to be ordered in such a way that the automatic creation of batches in Tensorflow results exactly in the way we want. For that is it is necessary to:
- A sequence needs to have 
- 

In [ ]:
BATCH_SIZE = 32 

tf.convert_to_tensor(non_fraud_seq_list[0][1])